# Anàlisi comparativa d'algorismes d'imputació: PSO, MICE i KNN
Aquest notebook està creat per poder generar gràfics i fer un anàlisi exhaustiva dels resultats obtinguts amb l'algorisme d'imputació Particle Swarm Optimization (PSO), fent una comparació amb dos algorismes: MICE i KNN. S'avalua el rendiment de cada mètode en diferents conjunts de dades de diverses mides, utilitzant mètriques com el fitness i el temps d'execució. L'objectiu és identificar quin algorisme ofereix millors resultats segons el context i la mida del conjunt de dades.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [ ]:
files = {
    'PSO': {
        'small': 'PSOResultsSmall.xlsx',
        'medium': 'PSOResultsMedium.xlsx',
        'big': 'PSOResultsBig.xlsx',
    },
    'KNN': {
        'small': 'KNNResultsSmall.xlsx',
        'medium': 'KNNResultsMedium.xlsx',
        'big': 'KNNResultsBig.xlsx',
    },
    'MICE': {
        'small': 'MICEResultsSmall.xlsx',
        'medium': 'MICEResultsMedium.xlsx',
        'big': 'MICEResultsBig.xlsx',
    }
}

dataframes = {}
for method, sizes in files.items():
    dataframes[method] = {}
    for size, path in sizes.items():
        xls = pd.ExcelFile(path)
        sheet_names = xls.sheet_names[:5]
        dfs = [xls.parse(sheet) for sheet in sheet_names]
        dataframes[method][size] = dfs

In [ ]:

def normalize_dataframe(df_list, method_name):
    all_rows = []
    for df in df_list:
        df = df.copy()
        df['method'] = method_name
        df.rename(columns={'small': 'dataset'}, inplace=True) 
        all_rows.append(df[['dataset', 'rows', 'columns', 'missing_percent', 'time_sec', 'fitness', 'method']])
    return pd.concat(all_rows, ignore_index=True)

normalized_dfs = []
for method in ['PSO', 'KNN', 'MICE']:
    for size in ['small', 'medium', 'big']:
        df = normalize_dataframe(dataframes[method][size], method_name=method)
        df['size'] = size
        normalized_dfs.append(df)

df_all = pd.concat(normalized_dfs, ignore_index=True)

dataset_map = {
    'test_0_rows_500_columns_20_comp_0.5_input_': 'Petit A',
    'test_0_rows_500_columns_20_comp_0.4_input_': 'Petit B',
    'test_0_rows_500_columns_20_comp_0.3_input_': 'Petit C',
    'test_0_rows_500_columns_20_comp_0.2_input_': 'Petit D',
    'test_0_rows_500_columns_100_comp_0.5_input_': 'Petit E',
    'test_0_rows_500_columns_100_comp_0.4_input_': 'Petit F',
    'test_0_rows_500_columns_100_comp_0.3_input_': 'Petit G',
    'test_0_rows_500_columns_100_comp_0.2_input_': 'Petit H',
    'test_0_rows_1000_columns_500_comp_0.5_input_': 'Mitjà J',
    'test_0_rows_1000_columns_500_comp_0.4_input_': 'Mitjà K',
    'test_0_rows_5000_columns_500_comp_0.5_input_': 'Gran L',
    'test_0_rows_5000_columns_500_comp_0.3_input_': 'Gran M',
}

df_all['dataset_name'] = df_all['dataset'].map(dataset_map)

summary = df_all.groupby(['dataset_name', 'method']).agg(
    fitness_mean=('fitness', 'mean'),
    fitness_std=('fitness', 'std'),
    time_mean=('time_sec', 'mean'),
    time_std=('time_sec', 'std')
).reset_index()

In [ ]:
os.mkdir('figures')

In [ ]:
def plot_grouped_bar_by_size(df_subset, metric_column, ylabel, title_prefix, filename):
    datasets = df_subset['dataset_name'].unique()
    methods = ['PSO', 'KNN', 'MICE']
    bar_width = 0.25
    x = range(len(datasets))

    plt.figure(figsize=(10, 6))
    for idx, method in enumerate(methods):
        subset = df_subset[df_subset['method'] == method]
        heights = subset[metric_column]
        positions = [p + bar_width * idx for p in x]
        plt.bar(positions, heights, width=bar_width, label=method)

    plt.xlabel('Dataset')
    plt.ylabel(ylabel)
    plt.title(f'{title_prefix}: Comparació per Mètode')
    plt.xticks([p + bar_width for p in x], datasets, rotation=45)
    plt.legend()
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(f"figures/{filename}", dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
df_petit = summary[summary['dataset_name'].str.contains("Petit")]
df_mitja = summary[summary['dataset_name'].str.contains("Mitjà")]
df_gran = summary[summary['dataset_name'].str.contains("Gran")]

In [ ]:
plot_grouped_bar_by_size(df_petit, 'fitness_mean', 'Fitness mitjà', 'Conjunts Petits',
                         'barplot_fitness_mean_per_method_dataset_petit.png')
plot_grouped_bar_by_size(df_mitja, 'fitness_mean', 'Fitness mitjà', 'Conjunts Mitjans',
                         'barplot_fitness_mean_per_method_dataset_mitja.png')
plot_grouped_bar_by_size(df_gran, 'fitness_mean', 'Fitness mitjà', 'Conjunts Grans',
                         'barplot_fitness_mean_per_method_dataset_gran.png')

In [ ]:
plot_grouped_bar_by_size(df_petit, 'time_mean', 'Temps d’execució mitjà (s)', 'Conjunts Petits',
                         'barplot_time_mean_per_method_dataset_petit.png')
plot_grouped_bar_by_size(df_mitja, 'time_mean', 'Temps d’execució mitjà (s)', 'Conjunts Mitjans',
                         'barplot_time_mean_per_method_dataset_mitja.png')
plot_grouped_bar_by_size(df_gran, 'time_mean', 'Temps d’execució mitjà (s)', 'Conjunts Grans',
                         'barplot_time_mean_per_method_dataset_gran.png')


In [ ]:
plot_grouped_bar_by_size(summary, 'fitness_mean', 'Fitness mitjà', 'Tots els conjunts',
                         'barplot_fitness_mean_per_method_dataset.png')

plot_grouped_bar_by_size(summary, 'time_mean', 'Temps mitjà d’execució (s)', 'Tots els conjunts',
                         'barplot_time_mean_per_method_dataset.png')

In [ ]:
def classify_size(row):
    if 'Gran' in row['dataset_name']:
        return 'Gran'
    elif 'Mitjà' in row['dataset_name']:
        return 'Mitjà'
    elif 'Petit' in row['dataset_name']:
        return 'Petit'
    else:
        return 'Altres'

df_all['dataset_name'] = df_all['dataset'].map(dataset_map)
df_all = df_all[df_all['dataset_name'].notna()]
df_all['size_group'] = df_all.apply(classify_size, axis=1)

In [ ]:
for size_group in ['Petit', 'Mitjà', 'Gran']:
    plt.figure(figsize=(10, 6))
    subset = df_all[df_all['size_group'] == size_group]
    sns.boxplot(data=subset, x='method', y='fitness')
    plt.title(f'{size_group}: Distribució del Fitness per Mètode')
    plt.xlabel('Mètode d’imputació')
    plt.ylabel('Fitness')
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(f"figures/boxplot_fitness_{size_group.lower()}.png", dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
for size_group in ['Petit', 'Mitjà', 'Gran']:
    plt.figure(figsize=(10, 6))
    subset = df_all[df_all['size_group'] == size_group]
    sns.scatterplot(data=subset, x='time_sec', y='fitness', hue='method', style='method')
    plt.title(f'{size_group}: Temps vs Fitness per Mètode')
    plt.xlabel('Temps d’execució (s)')
    plt.ylabel('Fitness')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"figures/scatter_time_vs_fitness_{size_group.lower()}.png", dpi=300, bbox_inches="tight")
    plt.show()


In [ ]:
def plot_bar_with_error_robust(df_subset, value_col, std_col, ylabel, title_prefix, filename):
    df_subset = df_subset[df_subset[std_col].notna()]
    datasets = df_subset['dataset_name'].unique()
    methods = ['PSO', 'KNN', 'MICE']
    bar_width = 0.25
    x = range(len(datasets))

    plt.figure(figsize=(10, 6))
    for idx, method in enumerate(methods):
        subset = df_subset[df_subset['method'] == method]
        if subset.empty:
            continue
        heights = subset[value_col].values
        stds = subset[std_col].values
        positions = [p + bar_width * idx for p in range(len(subset))]
        plt.bar(positions, heights, yerr=stds, width=bar_width, capsize=5, label=method)

    plt.xlabel('Dataset')
    plt.ylabel(ylabel)
    plt.title(f'{title_prefix}: Mitjana ± Desviació Estàndard')
    plt.xticks([p + bar_width for p in range(len(datasets))], datasets, rotation=45)
    plt.legend()
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(f"figures/{filename}", dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
plot_bar_with_error_robust(df_petit, 'fitness_mean', 'fitness_std', 'Fitness', 'Conjunts Petits',
                           'barplot_fitness_errorbars_petit.png')
plot_bar_with_error_robust(df_mitja, 'fitness_mean', 'fitness_std', 'Fitness', 'Conjunts Mitjans',
                           'barplot_fitness_errorbars_mitja.png')
plot_bar_with_error_robust(df_gran, 'fitness_mean', 'fitness_std', 'Fitness', 'Conjunts Grans',
                           'barplot_fitness_errorbars_gran.png')

In [ ]:
plot_bar_with_error_robust(df_petit, 'time_mean', 'time_std', 'Temps d’execució (s)', 'Conjunts Petits',
                           'barplot_time_errorbars_petit.png')
plot_bar_with_error_robust(df_mitja, 'time_mean', 'time_std', 'Temps d’execució (s)', 'Conjunts Mitjans',
                           'barplot_time_errorbars_mitja.png')
plot_bar_with_error_robust(df_gran, 'time_mean', 'time_std', 'Temps d’execució (s)', 'Conjunts Grans',
                           'barplot_time_errorbars_gran.png')

In [ ]:
subset_mitja = df_all[df_all['size_group'] == 'Mitjà']
plt.figure(figsize=(10, 6))
sns.boxplot(data=subset_mitja, x='method', y='fitness')
plt.title('Mitjà: Distribució del Fitness per Mètode')
plt.xlabel('Mètode d’imputació')
plt.ylabel('Fitness')
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("figures/boxplot_fitness_mitja.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=subset_mitja, x='time_sec', y='fitness', hue='method', style='method')
plt.title('Mitjà: Temps vs Fitness per Mètode')
plt.xlabel('Temps d’execució (s)')
plt.ylabel('Fitness')
plt.grid(True)
plt.tight_layout()
plt.savefig("figures/scatter_time_vs_fitness_mitja.png", dpi=300, bbox_inches="tight")
plt.show()